Get data from github

In [ ]:
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/train.csv
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/valid.csv
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/test.csv
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/bert_classifier.py
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/bert_dataset.py
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/requirements.txt

Installing Packages

In [3]:
import os 
print('Working in ', os.getcwd())

!pip install -r requirements.txt

Working in  d:\Documents\medicines-classification\bert_model\bert4classification
     -------------------------------------- 190.5/190.5 MB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 920.9 kB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 951.0 kB/s eta 0:00:00
     ---------------------------------------- 9.0/9.0 MB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 6.6/6.6 MB 1.1 MB/s eta 0:00:00
     ------------------------------------ 880.6/880.6 kB 899.2 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 2.0/2.0 MB 1.0 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895254 sha256=652dd86d8eb4daafc2dac351f1e0eaa5d43b68d2577efdcd684e7387a7a7e5f4
  Stored in directory: c:\users\анна\appdata\local\pip\cache\wheels\82\ab\9b\c15899bf659ba74f623ac776e8

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1 requires torch==1.12.1, but you have torch 1.8.1 which is incompatible.

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd

## Getting prepared train, valid data 

In [4]:
train_data = pd.read_csv('/content/train.csv')
valid_data = pd.read_csv('/content/valid.csv')
test_data  = pd.read_csv('/content/test.csv')

In [5]:
from bert_dataset import CustomDataset
from bert_classifier import BertClassifier

c:\Python\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initialize BERT classifier

In [6]:
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=2,
        epochs=2,
        model_save_path='pt_model/bert.pt'
)

Downloading: 100%|██████████| 632/632 [00:00<00:00, 317kB/s]
Downloading: 100%|██████████| 47.7M/47.7M [00:42<00:00, 1.11MB/s]
Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSeq

Prepare data and helpers for train and evlauation

In [7]:
classifier.preparation(
        X_train=list(train_data['text']),
        y_train=list(train_data['label']),
        X_valid=list(valid_data['text']),
        y_valid=list(valid_data['label'])
    )

Train loop

In [8]:
classifier.train()

Epoch 1/2
Train loss 0.7945727040289298 accuracy 0.6834731168148825
Val loss 0.8699987335717541 accuracy 0.7110332749562172
----------
Epoch 2/2
Train loss 0.8046949874838054 accuracy 0.7415205588744672
Val loss 0.9018999434438866 accuracy 0.717338003502627
----------


Check test data

In [9]:
texts = list(test_data['text'])
labels = list(test_data['label'])

predictions = [classifier.predict(t) for t in texts]

In [10]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.7257295950143019, recall: 0.7188798418940976, f1score: 0.7204235791808031
